In [316]:
import random
import math

prime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

Iмовiрнiсний тест Мiллера-Рабiна та допоміжна функція перевірки числа на псевдопростоту:

In [317]:
def pseudo_prime(x, d, p, s):
    if (x ** d) % p == 1 or (x ** d) % p == -1 % p:
        return 1
    x_r = x ** (2 * d) % p
    for i in range(1, s):
        if x_r == -1 % p:
            return 1
        if x_r == 1:
            return 0
        x_r = x_r**2 % p
    return 0

def miller_rabin_primality(p, k):
    j = 1
    while j < k:
        if p % 2 == 0:
            return "P is composite"
        n = p - 1
        s = 0
        while n % 2 == 0:
            n = n // 2
            s = s + 1
        x = random.randint(2, p - 1)
        if math.gcd(x, p) > 1:
            return "P is composite"
        if pseudo_prime(x, n, p, s) == 0:
            return "P is composite"
        j = j + 1
    return 1

Метод пробних ділень:

In [318]:
def probni_dilenya(n): # n = a_t * 10^t + a_(t-1) * 10^(t-1) + ... + a_1 * 10 + a0 * 1
    n_string = str(n)
    b = []
    for i in n_string[::-1]:
        b.append(int(i))
    
    for m in prime:
        r = [0] * len(b)
        r[0] = 1
        for i in range(len(b) - 1):
            r[i + 1] = r[i] * 10 % m   
        s = 0
        for i in range(len(b)):
            s += b[i] * r[i] 
        if s % m == 0:
            return m 
        
    return 1


$ \rho $ - метод Полларда:

In [319]:
def f(x, n):
    return (x ** 2 + 1) % n

def rho_pollard(n, x_0, f):
    X_val = {}
    X = [x_0]
    Y = [x_0]
    X_val[X[0]] = 1
    i = 0
    while X_val[X[i]] != 2:
        X.append(f(X[i], n))
        Y.append(f(f(Y[i], n), n))
        print(f"x_i: {X[i + 1]}, y_i: {Y[i + 1]}")
        if X[i + 1] not in X_val:
            X_val[X[i + 1]] = 1
        else:
            X_val[X[i + 1]] = X_val[X[i + 1]] + 1
        d = math.gcd(Y[i + 1] - X[i + 1], n)

        if X[i + 1] == Y[i + 1]:
            return 0
        if d != 1:
            return d
        i = i + 1
        
    return 0

Метод Брілхарта-Моріссона(CFRAC) і допоміжна функція(символ Лежандра)

In [320]:
def legendre(a, p):
    i = 1
    while True:
        while a >= p:
            a = a % p
        if a == 1 or a == 0:
            return a * i
        if a % 2 == 0:
            a = a // 2
            if p % 8 == 3 or p % 8 == 5:
                i = -i
        if a % 4 == 3 and p % 4 == 3:
            i = -i
        if a == 1:
            return a * i
        a, p = p, a

def cfrac(n):
    k = 10
    m = math.sqrt(n)

    a = [int(m)]
    u = [int(m)]

    v = [1]
    b = [1, a[0]]
    for i in range(k+1): ####
        v.append((n - u[i] ** 2) // v[i])
        a.append(((int(m) + u[i]) // v[i + 1]) // 1)
        u.append(a[i + 1] * v[i + 1] - u[i])
        b.append((b[i + 1] * a[i + 1] + b[i]) % n )
    return a, b

a, b = cfrac(17873)
print(a)
print(b)


[133, 1, 2, 4, 2, 3, 1, 2, 1, 2, 3, 3]
[1, 133, 134, 401, 1738, 3877, 13369, 17246, 12115, 11488, 17218, 9523, 10041]
